# Generate Hi-C matrices from GSE99363

by Shiwei Liu and Pu Zheng

2022.05.03

Data from: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE99363

Citation: https://www.nature.com/articles/ng.3906

In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

9688


# 0. load Hi-C data

## 0.1 load HiC raw counts

In [3]:
hic_raw_folder = r'\\10.245.74.212\Chromatin_NAS_3\HiC_cortex'
hic_raw_fl = os.path.join(hic_raw_folder, 'hic_raw_map.npy')

## 0.2 load HiC raw regions

In [4]:
%%time
# If load hic raw map (# n-by-3 array as bin-i; bin-j; contact count)
hic_raw_contacts = np.load(hic_raw_fl) # array as bin-i; bin-j; contact count

# If load hic region annotation (# n-by-4 array as chr;start;end;bin-name)
hic_reg_savename = os.path.join(hic_raw_folder, "hic_reg_annotation.npy")
hic_raw_regs = np.load(hic_reg_savename)

print(hic_raw_regs.shape)  # array as chr;start;end;bin-name

(250885, 4)
Wall time: 2.81 s


* This raw regions have bug in sorting, therefore add extra layer of filtering

In [5]:
hic_sel_regs = []
curr_reg = 0
curr_start = -1
for _reg_info in hic_raw_regs:
    #print(_reg_info[-1], curr_reg)
    if _reg_info[-1] != curr_reg and _reg_info[1] > curr_start:
        hic_sel_regs.append(_reg_info)
        curr_reg = _reg_info[-1]
        curr_start = _reg_info[1]
    elif _reg_info[-1] != curr_reg and hic_sel_regs[-1][0] != _reg_info[0]:
        hic_sel_regs.append(_reg_info)
        curr_reg = _reg_info[-1]
        curr_start = _reg_info[1]
    else:
        continue
hic_sel_regs = np.array(hic_sel_regs)
# Convert into pandas.DataFrame
import pandas as pd
hic_reg_df = pd.DataFrame(hic_sel_regs.astype(np.int32),columns=['chr','start','end','name'])

In [7]:
from tqdm import tqdm
chr_2_contact = {}
chr_2_regDf = {}
for _chr in np.unique(hic_reg_df['chr']):
    # select specific chr
    print(_chr)
    _chr_reg_df = hic_reg_df.loc[hic_reg_df['chr']==_chr].sort_values('start')
    _chr_contacts = np.zeros([len(_chr_reg_df),len(_chr_reg_df)], dtype=np.int32)
    print(_chr_contacts.shape)
    _chr_reg_lst = list(_chr_reg_df['name'].values)
    _chr_reg_min, _chr_reg_max = np.min(_chr_reg_lst), np.max(_chr_reg_lst)
    # loop through contacts
    for _bi, _bj, _ct in tqdm(hic_raw_contacts):
        if _bi >= _chr_reg_min and _bi <= _chr_reg_max and _bj >= _chr_reg_min and _bj <= _chr_reg_max:
            _bi, _bj = int(_bi),int(_bj)
            if _bi in _chr_reg_lst and _bj in _chr_reg_lst:
                _chr_contacts[_chr_reg_lst.index(_bi), _chr_reg_lst.index(_bj)] += _ct
    chr_2_contact[_chr] = _chr_contacts
    chr_2_regDf[_chr] = _chr_reg_df

1
(9774, 9774)


100%|██████████████████████████████████████████████████████████████████| 37155393/37155393 [1:07:47<00:00, 9134.55it/s]


2
(9106, 9106)


100%|█████████████████████████████████████████████████████████████████| 37155393/37155393 [1:00:38<00:00, 10212.39it/s]


3
(8002, 8002)


...

100%|███████████████████████████████████████████████████████████████████| 37155393/37155393 [13:07<00:00, 47190.93it/s]


19
(3072, 3072)


100%|███████████████████████████████████████████████████████████████████| 37155393/37155393 [06:41<00:00, 92652.90it/s]


20
(4588, 4588)


100%|██████████████████████████████████████████████████████████████████| 37155393/37155393 [01:18<00:00, 474714.04it/s]


In [ ]:
# Save regions

In [ ]:
hic_reg_df.to_csv(os.path.join(hic_raw_folder, 'filtered_regions.csv'), 
                  index=False)